In [33]:
import numpy as np

from keras.models import Model, Sequential
from keras.layers import Dropout, LSTM, Activation, Dense
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


In [36]:
sentence_length = 100 
file_name = "sherlock_clean.txt"

int_to_char = {}
char_to_int = {}
epochs = 20
batch = 80

def prepare_dataset(file_name, seq_length):
    raw_text = open(file_name).read().lower()
    unique_chars = list(set(raw_text))
    #unique_chars = [c for c in unique_chars if str.isalnum(c)]
    n_count = len(raw_text)
    n_uniq_char = len(unique_chars)
    print("Total characters in file",n_count)
    print("Total unique characters in file ",n_uniq_char)
    char_to_int, int_to_char = get_lookups(unique_chars)
    
    sentences = []
    labels = []
    for i in range(0, len(raw_text)-seq_length, 1):
        phrase = raw_text[i:seq_length]
        label_char = raw_text[i+seq_length]
        sentences.append(phrase)
        labels.append(label_char)
    
    n_sentences = len(sentences)
    n_labels = len(labels)
    
    #vectorization, create 3D matrix that LSTM can consume
    #each data item has seq_length characters, each of which is encoded as 1-hot of length n_uniq_char
    X = np.zeros( (n_sentences, seq_length, n_uniq_char) , dtype = np.bool)
    y = np.zeros( (n_labels, n_uniq_char) , dtype = np.bool)
    for s, sent in enumerate(sentences):
        for t, c in enumerate(sent):
            X[s, t, char_to_int[c]] = 1
        y[s, char_to_int[labels[s]]] = 1
    
    return char_to_int, int_to_char, sentences, labels, X, y

def get_lookups(uniq_chars):
    char_to_int = {c:ind for ind, c in enumerate(uniq_chars) }
    int_to_char = { ind:char for char,ind in char_to_int.items()}
    print("\n")
    print("Char lookup ",char_to_int,)
    print("\n")
    print("Int lookup ",int_to_char,)
    return char_to_int, int_to_char


def get_model(n_hidden, X, y, epochs, batch):
    
    model = Sequential()
    model.add( LSTM(n_hidden, input_shape = (X.shape[1], X.shape[2])) )
    model.add( Dropout(0.25) )
    model.add( Dense(y.shape[1], activation = "softmax") )

    model.compile(loss = "categorical_crossentropy", optimizer = "adam")
    model.summary()
    filepath="weights-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    
    model.fit(X, y, epochs=epochs, batch_size=batch , callbacks=callbacks_list)
    
    return model

char_to_int, int_to_char, sentencs, labels, X, y  = prepare_dataset(file_name,sentence_length)
model = get_model(256, X, y, epochs, batch)

Total characters in file 562332
Total unique characters in file  55


Char lookup  {'f': 0, 'c': 1, '"': 2, 'u': 3, 'e': 4, 'p': 5, 'a': 6, '-': 7, 'y': 8, 'q': 9, 'g': 10, '?': 11, '6': 12, ' ': 13, ':': 14, '8': 15, 's': 16, '(': 17, 'z': 18, 'm': 19, ',': 20, 'k': 21, 'v': 22, 'è': 23, 'l': 24, '5': 25, 'à': 26, '9': 27, '3': 28, 'x': 29, 'h': 30, 'o': 31, '!': 32, 'â': 33, 'é': 34, '&': 35, '7': 36, ';': 37, 'j': 38, 'n': 39, 't': 40, '4': 41, 'i': 42, '1': 43, '0': 44, ')': 45, 'd': 46, '\n': 47, 'b': 48, '2': 49, 'r': 50, '/': 51, 'w': 52, "'": 53, '.': 54}


Int lookup  {0: 'f', 1: 'c', 2: '"', 3: 'u', 4: 'e', 5: 'p', 6: 'a', 7: '-', 8: 'y', 9: 'q', 10: 'g', 11: '?', 12: '6', 13: ' ', 14: ':', 15: '8', 16: 's', 17: '(', 18: 'z', 19: 'm', 20: ',', 21: 'k', 22: 'v', 23: 'è', 24: 'l', 25: '5', 26: 'à', 27: '9', 28: '3', 29: 'x', 30: 'h', 31: 'o', 32: '!', 33: 'â', 34: 'é', 35: '&', 36: '7', 37: ';', 38: 'j', 39: 'n', 40: 't', 41: '4', 42: 'i', 43: '1', 44: '0', 45: ')', 46: 'd', 47

KeyboardInterrupt: 